NOTE: because of the way remapping works for TMAs, we recommend running this notebook cell-by-cell rather than all at once.

In [ ]:
import json
import os
from skimage.io import imread

from ark.mibi import tiling_utils

# suppress mpl deprecation
import warnings
from matplotlib.cbook import mplDeprecation
warnings.filterwarnings("ignore", category=mplDeprecation)

# 1. Automatically generate tiled regions

This automatically creates tiles which define FOVs on the MIBI slide based on user-set parameters.

### Define paths to the JSON data

In [ ]:
base_dir = "../data/example_dataset"
json_tiling_dir = os.path.join(base_dir, "json_tiling_data")
fov_list_path = os.path.join(json_tiling_dir, 'fov_list_single_fov_tma.json')
moly_path = os.path.join(json_tiling_dir, 'moly_point.json')
tiled_output_path = os.path.join(json_tiling_dir, 'tiled_output_tma.json')

### Define whether TMA is used or not

In [ ]:
tma = True

### Set tiling parameters

The following parameters will be defined:

* Starting x coordinate and y coordinate for each fov (taken from `fov_list_file`)
* Number of fovs along the x- and y-axis for each fov (set by user)
* x- and y-axis step size (set by user)
* Whether to randomize the order of the fovs or not (set by user)

In [ ]:
# automatically define upper-right and bottom-left hand corner for TMAs, which then autogenerates the tiles in between
tiling_params, moly_point = tiling_utils.set_tiling_params(
    fov_list_path,
    moly_path,
    tma=tma
)

### Create the FOV tiles for each region

In [ ]:
tiled_regions = tiling_utils.create_tiled_regions(
    tiling_params,
    moly_point,
    tma=tma
)

In [ ]:
# write the tiled output
with open(tiled_output_path, 'w') as top:
    json.dump(tiled_regions, top)

# 2. Slide overlay QC (for TMAs only)

TODO: probably will separate this to a different notebook

Using a pre-defined JSON of FOVs, compares with the automatically-generated FOVs from the previous step and allows for remapping if necessary.

### Define QC parameters, read data in

Define the following parameters:

* `proposed_tiled_output_path`: the path to your proposed set of tiles
* `slide_path`: the path to the slide which to take the tiles
* `mapping_path`: the path which to save the final mapping of your proposed tiles to the tiles generated by the script (contained in `tiled_regions`)

NOTE: the data contained in `proposed_tiled_output_path` is assumed to have the same tile dimensions and the same Moly point as the data in `tiled_output_path`.

In [ ]:
proposed_tiled_output_path = os.path.join(json_tiling_dir, 'noah_test_tiles.json')
slide_path = os.path.join(json_tiling_dir, 'noah_test_slide.png')
mapping_path = os.path.join(json_tiling_dir, 'proposed_auto_map.json')

In [ ]:
# load the proposed set of tiles in
with open(proposed_tiled_output_path, 'r') as ptop:
    tiled_regions_proposed = json.load(ptop)

In [ ]:
# load the slide image in
slide_data = imread(slide_path)

### Map proposed tiles to their closest automatically-generated tile (Euclidean)

In [ ]:
proposed_to_auto_map, proposed_tiles_info, auto_tiles_info = tiling_utils.assign_closest_tiled_regions(
    tiled_regions_proposed,
    tiled_regions,
    moly_point['name']
)

### Visualize and remap tiles

Usage notes:

* Proposed tiles are drawn in black. Automatically-generated tiles are drawn in blue.
* The selected proposed tile and its mapped automatically-generated tile are bolded.
* The `Proposed tile` tab can be used to visualize current mappings to automatically-generated tiles.
* The `Automatically-generated tile` tab is used for re-mapping a tile created by this script to one of your's.

In [11]:
%matplotlib widget
tiling_utils.interactive_remap(
    proposed_to_auto_map,
    proposed_tiles_info,
    auto_tiles_info,
    slide_data,
    mapping_path,
    draw_radius=10,
    figsize=(12, 12)
)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Output()

### Use mapping to rename FOVs in `tiled_regions_proposed`

In [ ]:
# load the mapping saved in the interactive visualization in
with open(mapping_path, 'r') as mp:
    mapping = json.load(mp)

In [ ]:
# use the mapping to rename each FOV in tiled_regions_proposed to its corresponding automatically-generated tile name
for fov in tiled_regions_proposed['fovs']:
    fov['name'] = mapping[fov['name']]

In [ ]:
# resave tiled_regions_proposed with the new FOV names
with open(proposed_tiled_output_path, 'w') as ptop:
    json.dump(tiled_regions_proposed, ptop)